# Tutorial 10 - Object Detection with YOLO on our Dataset

## Dr. David C. Schedl

## Setup
As first step, we need to import the necessary libraries.


In [ ]:

!curl -LJO "https://raw.githubusercontent.com/Digital-Media/cv_data/main/fhhgb-hockey-dataset.zip" --silent
import zipfile
import os

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Extract the dataset to the data directory
with zipfile.ZipFile("fhhgb-hockey-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

print("Dataset extracted to data/ directory")

## Loading a Pre-trained Model

We'll start by loading a pre-trained YOLO model that has been trained on the FHHGB Hockey Dataset. This model can detect hockey nets and pucks in hockey game footage.


In [ ]:
# Setup and import of libraries
try:
  import ultralytics
except:
  %pip install ultralytics --quiet

from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os



# Load a pretrained YOLO model trained on hockey dataset
model = YOLO("./data/FHHGB-Hockey-Dataset/models/weights.pt")

print("Model loaded successfully!")
print(f"Model classes: {model.names}")


# Testing on a Single Image

Let's test our model on a single image from the test set to see how it performs. We'll use one of the hockey images to demonstrate the object detection capabilities.


In [ ]:
# Select a test image from the hockey dataset
test_image_path = "data/FHHGB-Hockey-Dataset/test/images/data1-12_png_jpg.rf.ec74890a35a41e81fa73e82288f71d83.jpg"

# Verify the image exists
if os.path.exists(test_image_path):
  print(f"Using test image: {test_image_path}")

  # Perform object detection on the test image
  results = model(test_image_path)

  print(f"Detection completed on: {test_image_path}")

# Visualizing Results

Now let's visualize the detection results. The model will draw bounding boxes around detected objects and label them with their class names and confidence scores.


In [ ]:
# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Display the image with detections
    plt.figure(figsize=(12, 8))
    plt.imshow(im_rgb)
    plt.title(f"Object Detection Results - Image {i+1}")
    plt.axis('off')
    plt.show()

    # Show results to screen (in supported environments)
    # r.show()

    # Save results to disk
    r.save(filename=f"results{i}.jpg")
    print(f"Results saved as: results{i}.jpg")


### Analyzing Detection Results

Let's examine the detection results in more detail to understand what objects were detected and their confidence scores.


In [ ]:
# Analyze detection results
for i, r in enumerate(results):
    print(f"\n=== Detection Results for Image {i+1} ===")

    if r.boxes is not None:
        # Get detection information
        boxes = r.boxes

        print(f"Number of detections: {len(boxes)}")

        for j, box in enumerate(boxes):
            # Get class name and confidence
            cls = int(box.cls[0])
            conf = float(box.conf[0])

            # Get class name from model names
            class_name = model.names[cls]

            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

            print(f"  Detection {j+1}: {class_name} (confidence: {conf:.3f})")
            print(f"    Bounding box: ({x1:.1f}, {y1:.1f}) to ({x2:.1f}, {y2:.1f})")
    else:
        print("No objects detected in this image.")


# Model Validation

Now let's evaluate the model's performance on the validation set to get quantitative metrics like precision, recall, and mAP (mean Average Precision).


In [ ]:
# Validate on validation set
print("Running validation on the validation set...")
metrics_val = model.val(data="data/FHHGB-Hockey-Dataset/data.yaml", split="val")

print("\nResults saved to the 'runs/detect/' directory")



# Understanding the Results

## Performance Metrics Explained

- **mAP@0.5**: Mean Average Precision at IoU threshold of 0.5
- **mAP@0.5:0.95**: Mean Average Precision averaged over IoU thresholds from 0.5 to 0.95
- **Precision**: Ratio of true positive detections to all positive detections
- **Recall**: Ratio of true positive detections to all actual objects

## Class-Specific Performance

The model shows different performance for different classes:
- **Net detection**: Generally high accuracy due to its large, distinctive appearance
- **Puck detection**: Lower accuracy due to its small size and fast movement

## Practical Applications

This hockey object detection model could be used for:
- Automated game analysis
- Player tracking systems
- Goal detection and verification
- Training data generation for sports analytics


# 📝 Exercise

Try the following exercises to better understand object detection:

1. **Test on different images**: Try running the model on other test images from the dataset
2. **Analyze failure cases**: Look for images where the model fails to detect objects correctly
3. **Confidence thresholding**: Experiment with different confidence thresholds to see how they affect detection results
4. **Real-time detection**: If you have a webcam, try running the model on live video feed

## Useful Links

- [Ultralytics YOLO Documentation](https://docs.ultralytics.com/)
- [YOLO Paper](https://arxiv.org/abs/1506.02640)
- [Object Detection Tutorial](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)
